In [1]:
import copy
import random
import time
import os
import re

import torch
import torch.nn as nn
import torch.nn.functional 
import torch.optim 
import torch.utils.data

import torchvision.transforms
import torchvision.datasets

import skimage.io
import skimage.transform
import sklearn.preprocessing

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pd.options.mode.chained_assignment = None

# Functions

In [3]:
def set_seeds(seed):
    """sets seeds for several used packages"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [4]:
def encode_column(column):
    """
    takes single columned Pandas DataFrame of categorical data and encodes it
    into array of class binarys
    """
    encoder = sklearn.preprocessing.OneHotEncoder()
    shape_arr = encoder.fit_transform(column).toarray().astype(int)
        
    return list(shape_arr)

In [5]:
def prep_data(labels, image_root):
    """
    Takes in raw labels dataframe and converts it into the format
    expected for tenX_dataset class
    """

    #Splitting description column into color and shape columns
    new = labels["Description"].str.split(" ", n=1, expand=True)
    labels.drop(columns=['Description'], inplace=True)
    labels['Color'] = new[0].values
    labels['Shape'] = new[1].values
    
    #Decomposing sample keywords into seperate strings
    sample_names = labels["Sample"].str.split(" ", n=1, expand=False)
    labels['Sample'] = sample_names
    
    #Converting identification into boolean for is/is not plastic
    PLASTICS = ['polystyrene', 'polyethylene','polypropylene','Nylon','ink + plastic','PET','carbon fiber','poly(ethylene glycol) monooleate or polyamide resin']
    identification = labels['Identification']
    
    for i in range(0,len(identification)):
        if identification[i] in PLASTICS:
            identification[i] = True
        else:
            identification[i] = False

    labels['Identification']=identification
    labels.rename(columns={'Identification': 'isPlastic'}, inplace=True)
    labels['isPlastic'] = labels["isPlastic"].astype(int)
    
    
    #Encoding shape and color data
    labels['Shape'] = encode_column(labels[['Shape']])
    labels['Color'] = encode_column(labels[['Color']])
    
    labels = add_filenames(labels, image_root)
    
    return labels

In [6]:
def add_filenames(labels, image_root):
    """
    Replaces sample column of labels with the actual filename so that the dataset class doesn't have to do that work.
    """
    image_filenames = os.listdir(image_root)
    labels.insert(loc=1, column='File', value=None)
    for index, row in labels.iterrows():
        sample = row['Sample']
        for fname in image_filenames:
            str_id = '^' + ' '.join(row['Sample']) + ' .*'
            result = re.search(str_id, fname)
            if result:
                image_file = result.group()
                assert(os.path.exists('./data/images_10x/' + image_file))
                break
        else:
            image_file = None
        labels.loc[index, 'File'] = image_file
    return labels

In [7]:
def remove_nones(df):
    """further preps dataframe by removing nones, should probably move into prep_data"""
    for index, row in df.iterrows():
        if row['File'] == None:
            df.drop(index, inplace=True)
    df = df.reset_index(drop=True)
    return df


In [8]:
def split_train_test(labels_df, train_prop):
    """
    Input: prepped and cleaned dataframe with image and corresponding
    plastic/non-plastic identifications, desired proportion for training set (decimal between 0-1) 
    Returns: training dataframe with train_prop of the total data, and test_df with the remaining data
    """
    split_ratio = 0.5
    train_df = (labels_frame.sample(frac = train_prop))
    test_df = (labels_frame.drop(train_df.index))

    # reset index to avoid KeyErrors!
    train_df = train_df.reset_index()
    test_df = test_df.reset_index()
    return train_df, test_df

In [9]:
# runs but may have bugs- net seems to only guess 1 or 0 no matter what 

def train_model(train_loader, network, optimizer, criterion, nepochs):
    """
    This function trains a convolutional neural network.
    Input: the training set data loader, defined network, optimizer , crtiterion, and number of epochs
    Returns: trained neural network 
    """
    for epoch in range(nepochs):  # loop over the dataset
        running_loss = 0.0
    
        for i, data in enumerate(train_loader, 0):

            #get the inputs and correspponding labels
            inputs = data['image']
            labels = data['plastic']

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs =  network(data['image'])
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            print('Finished Training')
    return network 


# Custom Dataset

In [10]:
class tenX_dataset(torch.utils.data.Dataset):
    """
    Class inherited from torch Dataset. Required methods are, init,
    len, and getitem.
    """
    def __init__(self, labels_frame, image_dir, transform):
        """
        initializes an instance of the class. Here we store 4 variables
        in the class. Calling init just looks like dataset = tenX_dataset(lables, 'image_folder', transform).
        
        labels: altered version of csv file
        image_dir: The file path to the folder the images are in
        image_filenames: A list of all the image file names in the image folder
        transform: A pytorch object. Works like a function. You call transform(x) and it performs
                    a series of operations on x
        """
        self.labels = labels_frame
        self.image_dir = image_dir
        self.image_filenames = os.listdir(self.image_dir)
        self.transform = transform
        

    def __len__(self):
        """Returns the length of the dataset"""
        return len(self.labels)
    
    
    def __getitem__(self, idx):
        """
        Returns a dictionary containing image and image data. Right now
        it looks like: 
        sample = {'image': image, 'plastic': [0], 'shape':[0,0,0,0,0], 'color':[0,0,0,0,0]}
        """
        image_filename = self.labels['File'][idx]
        image = None
             
        if image_filename is not None:
            image_filepath = os.path.join(self.image_dir, image_filename)
            image = skimage.io.imread(image_filepath)
            if self.transform is not None:
                image = self.transform(image)

        sample = {'image': image,
                  'shape': self.labels['Shape'][idx],
                  'color': self.labels['Color'][idx],
                  'plastic': self.labels['isPlastic'][idx]}
  
        return sample

In [11]:
def save_network(network, path):
    """
    Saves trained neural network to a file
    Input: trained network, path for saving (e.g. './trained_test1.pth')
    Returns: None, saves network to the specified file 
    """
    PATH = path
    torch.save(network.state_dict(), PATH)
    return None
    

### Plotting first 20 images of dataset. Obviously getting quite a few duplicates

In [12]:
# labels_filepath = 'data/10x_labels_more.csv'
# image_dir = 'data/images_10x'
# labels = prep_data(pd.read_csv(labels_filepath, delimiter='\t'), image_dir)
# tenX = tenX_dataset(labels, image_dir, None)


# for i in range(len(tenX)):
#     sample = tenX[i]
#     plt.figure(i)
#     if sample['image'] is not None:
#         plt.imshow(sample['image'])
#     if i>20:
#         break

# Things to improve/fix
* Make sure the nonetypes are because the file actually isn't in my folder of images
* Code for normalizing image data
* Image augmentation. Probably want to cut off some of the edges to get rid of number stuff and decrease extraneous information. The think we actually care about is only occupying like 5-10% of the image.

# CNN 

starting from simple test network lik eher:https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html


In [13]:
# set seeds - for reproducibility
set_seeds(10)

In [14]:
# define the transform for the image
transform = torchvision.transforms.Compose([torchvision.transforms.ToPILImage(),
                                             torchvision.transforms.CenterCrop((300, 350)),
                                             torchvision.transforms.RandomRotation((-180,180)),
                                             torchvision.transforms.ToTensor()
                                            ])

In [72]:
#load data and split into test and train DataFrames
image_dir = 'data/images_10x'
labels_frame = remove_nones(prep_data(pd.read_csv('data/10x_plastic_bias.csv', delimiter='\t'), image_dir))
train_df, test_df = split_train_test(labels_frame, 0.7)

In [153]:
train_set = tenX_dataset(train_df, image_dir, transform = transform)
test_set = tenX_dataset(test_df, image_dir, transform = transform)

batch_size = 5
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [99]:
print(len(train_set))
print(len(test_set))
print(len(trainloader))
print(len(testloader))

135
58
27
12


In [51]:
import torch.nn as nn
import torch.nn.functional


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        #self.pool = torch.nn.MaxPool2d(3, 2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fc1 = nn.Linear(72576, 6)
        self.fc2 = nn.Linear(6, 6)
        self.fc3 = nn.Linear(6,2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.max_pool2d(x, kernel_size = 2)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.max_pool2d(x, kernel_size = 2)
        x = nn.functional.relu(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)    
        x = nn.functional.relu(x)
        x = self.fc2(x)    
        x = nn.functional.relu(x)
        x = self.fc3(x)    
#         x = self.pool(torch.nn.functional.relu(self.conv1(x)))
#         x = self.pool(torch.nn.functional.relu(self.conv2(x)))
#         x = x.view(x.shape[0], -1)
#         x = nn.functional.relu(self.fc1(x))
#         x = nn.functional.relu(self.fc2(x))
#         x = self.fc3(x)
        return x
    


net = Net()

In [52]:
#defining loss function and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [54]:
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        
        inputs = data['image']
        labels = data['plastic']

        # zero the parameter gradients
        #optimizer.zero_grad()

        # forward + backward + optimize
        outputs =  net(data['image'])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #print(net.conv1.weight.grad) 
        #print(net.conv2.bias.grad[3])

        # print statistics
        #running_loss += loss.item()
        #if i % 2000 == 1999:    # print every 2000 mini-batches
        #    print('[%d, %5d] loss: %.3f' %
        #          (epoch + 1, i + 1, running_loss / 2000))
        #    running_loss = 0.0

print('Finished Training')

Finished Training


In [82]:
save_network(net, './trained_test1.pth')

dataiter = iter(testloader)
images = dataiter.next()['image']
labels = dataiter.next()['plastic']

outputs = net(images)
outputs

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images = data['image']
        labels = data['plastic']
        outputs = net(images)
        print(data['plastic'])
        predicted = torch.max(outputs.data, 1)[1]
        print('prediction ', predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (len(testloader),
    100 * correct / total))

tensor([1, 1, 0, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 1, 0, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 1, 0, 1, 1])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1])
prediction  tensor([0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 1, 0, 0, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1])
prediction  tensor([0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 1])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([0, 0, 1, 1, 1])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 0, 0])
prediction  tensor([0, 0, 0])
Accuracy of the network on the 12 test images: 50 %


tensor([0, 0, 1, 1, 1, 0, 0, 1, 0, 1])
prediction  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 1, 1, 0, 1, 0, 0, 1, 0, 1])
prediction  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 1])
prediction  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 1, 1, 0, 1, 0, 1, 0, 0])
prediction  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 1, 1, 1, 1, 0, 1, 0])
prediction  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 1, 0, 0, 1, 1, 0, 1])
prediction  tensor([0, 0, 0, 0, 0, 0, 0, 0])
Accuracy of the network on the 6 test images: 56 %


## try a different model 

In [162]:
class NetTwo(nn.Module):
    def __init__(self):
        super(NetTwo, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 5)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(193536, 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, 2)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1,193536 )
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(nn.functional.relu(self.fc2(x)))
        x = self.softmax(self.fc3(x))
        return x
    
model = NetTwo()

In [163]:
# defining the model
model = NetTwo()
# defining the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

NetTwo(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=193536, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
  (softmax): LogSoftmax()
)


In [170]:
train_set = tenX_dataset(train_df, image_dir, transform = transform)
test_set = tenX_dataset(test_df, image_dir, transform = transform)

batch_size = 1
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [175]:
model.eval()
# iterate over test data
test_loss = 0

len(testloader)
for i, sample in enumerate(trainloader):
    # forward pass
    data = sample['image']
    target = sample['plastic']
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) 
    # calculate test accuracy for each object class


In [80]:
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        
        inputs = data['image']
        labels = data['plastic']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs =  model(data['image'])
        print()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #print(net.conv1.weight.grad) 
        #print(net.conv2.bias.grad[3])

print('Finished Training')

Finished Training


In [81]:
save_network(model, './trained_test2.pth')

dataiter = iter(testloader)
images = dataiter.next()['image']
labels = dataiter.next()['plastic']

outputs = net(images)
outputs

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images = data['image']
        labels = data['plastic']
        outputs = net(images)
        print(data['plastic'])
        predicted = torch.max(outputs.data, 1)[1]
        print('prediction ', predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (len(testloader),
    100 * correct / total))

tensor([1, 1, 0, 0, 1])
prediction  tensor([0, 0, 0, 0, 0])
tensor([0, 1, 1, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 0, 1, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([0, 1, 0, 1, 1])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 0, 0, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 1, 0, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([0, 0, 0, 1, 1])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 0, 1, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([0, 1, 0, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([0, 1, 1, 1, 0])
prediction  tensor([0, 0, 0, 0, 0])
tensor([1, 0, 0])
prediction  tensor([0, 0, 0])
Accuracy of the network on the 12 test images: 50 %


# OLD

In [ ]:
#This transform just resizes the images to 3,480,752. So 3 for red green blue then height of 480
#and width of 752. 
#transform = torchvision.transforms.Compose([
 #                           torchvision.transforms.ToPILImage(),
 #                           torchvision.transforms.Resize((480, 752)),
   #                         torchvision.transforms.ToTensor()
  #                                    ])

In [ ]:
#define image directory and cleaned up dataframe
image_dir = 'data/images_10x'
labels_frame = remove_nones(prep_data(pd.read_csv('data/10x_plastic_bias.csv', delimiter='\t'), image_dir))


In [ ]:
#tenX = tenX_dataset(labels_frame, image_dir, transform = transform)

# Start of me trying to plug into cnn

Most of the code came from this tutorial: https://github.com/bentrevett/pytorch-image-classification/blob/master/2_lenet.ipynb

I was just trying to get this to work so I won't understand it as much

In [ ]:
image_dir = 'data/images_10x'
labels_frame = labels

#This transform just resizes the images to 3,480,752. So 3 for red green blue then height of 480
#and width of 752. 
transform = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.Resize((480, 752)),
                            torchvision.transforms.ToTensor()
                                      ])


train_data = tenX_dataset(labels_frame, image_dir, transform = transform)

In [ ]:
image_dir = 'data/images_10x'
labels_frame = remove_nones(prep_data(pd.read_csv('data/10x_labels_more.csv', delimiter='\t'), image_dir))
transform = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.Resize((480, 752)),
                            torchvision.transforms.ToTensor()
                                      ])


train_data = tenX_dataset(labels_frame, image_dir, transform = transform)

#### Splitting into train/validation set

In [ ]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = torch.utils.data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')

#### Declaring iterator. The thing that will loop through our dataset.

In [ ]:
BATCH_SIZE = 5

train_iterator = torch.utils.data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = torch.utils.data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

#### The CNN archetecture

In [ ]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        """
        Initializes CNN. Here we just define layer shapes that we call in the forward func
        """
        super().__init__()

        #Convulution layer 1. 
        #3 input channels (for three images Red, Green, Blue)
        #6 output channels (I THINK this means we are applying two different filters to each image
        #3 images, two filters each, we end up with 6 'images')
        #kernel size is I THINK telling the filters took filter each set of 5 pixels into one.
        #So are images will shrink a little as the edges get cut off
        self.conv1 = nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5)
        
        #Convultion layer 2. See above
        self.conv2 = nn.Conv2d(in_channels = 6, 
                               out_channels = 12, 
                               kernel_size = 5)
        
        #Linear layers. These probably arent complicated but I don't follow haha
        #I think it turning the 259740 pixel values into 6 values. Then the second layers
        #Turns the 6 into a different 6? and then 6 into 2. I'm not sure why 2 and not 1.
        #Seeing as the output should be a number between 0-1. Closer to 0 = not plastic,
        #closer to 1 = plastic. But I got errors about not having enough classes when
        #I only had 1 output neuron.
        #TBH these linear layers I just changed based on the error messages I got.
        self.fc_1 = nn.Linear(259740, 6)
        self.fc_2 = nn.Linear(6, 6)
        self.fc_3 = nn.Linear(6, 2)

    def forward(self, x):
        """
        Function that performs all the neural network forward calculation i.e.
        takes image data from the input of the neural network to the output
        """

        
        x = self.conv1(x)
    
        #Gonna have to look at tutorial link.
        x = nn.functional.max_pool2d(x, kernel_size = 2)
        
        x = nn.functional.relu(x)
        
        x = self.conv2(x)
                
        x = nn.functional.max_pool2d(x, kernel_size = 2)
        
        x = nn.functional.relu(x)
        
        x = x.view(x.shape[0], -1)
                
        h = x
        
        x = self.fc_1(x)
                
        x = nn.functional.relu(x)

        x = self.fc_2(x)
                
        x = nn.functional.relu(x)

        x = self.fc_3(x)
        
        return x, h

In [ ]:
#Instancing model, loss criteria, device to perform calculations on, and optimizer.
OUTPUT_DIM = 1
model = LeNet(OUTPUT_DIM)


criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
#Telling the model and loss function to do math on whatever device is
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def calculate_accuracy(y_pred, y):
    """
    Function calculate accuracy. See tutorial, may not
    even be accurate for our model but it at least runs
    """
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    """
    Training loop. Takes data through NN calculates loss and adjusts NN. Repeat
    """
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    #Need to add logic to skip iteration if image is None
    for sample in iterator:  
        
        image = sample['image'].to(device)
        isPlastic = sample['plastic'].to(device)
                
        optimizer.zero_grad()      
        y_pred, what = model(image)

        loss = criterion(y_pred, isPlastic)
        acc = calculate_accuracy(y_pred, isPlastic)
        loss.backward()    
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
#Here the model is actually trained
EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)

    
    end_time = time.monotonic()

    #epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    #print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')